In [1]:
# library

import random
import pandas as pd
import numpy as np
import os
import glob
import cv2
from PIL import Image
import matplotlib.patches as patches
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
# from torch.utils.tensorboard import SummaryWriter
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
from tqdm.auto import tqdm
from Baseline_func import *
from model import *

/home/jrkim/.conda/envs/intern/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# setting for classfication

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('device :', device)
print('Current :', torch.cuda.current_device())
print('Count :', torch.cuda.device_count())

cfg = {
    'img_size':512,
    'hidden_size':48,
    'learning_rate':0.0001,
    'num_epochs':100,
    'batch_size':32,
    'seed':30,
    'save_interval':5
}
seed_everything(cfg['seed'])
# data load
train_dataset = ClassDataset('./train', train=True, transforms=None, pad=70, img_size=cfg['img_size'])
train_loader = DataLoader(train_dataset, batch_size=cfg['batch_size'], shuffle=True, collate_fn=dynamic_collate_fn)
!nvidia-smi

device : cuda
Current : 0
Count : 1
Tue May 30 22:37:03 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.29.05    Driver Version: 495.29.05    CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:06:00.0 Off |                    0 |
| N/A   42C    P0    56W / 300W |   2982MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:07:00.0 O

In [3]:
# model, optimizer, scheduler for detector

model = Classifier(img_size=cfg['img_size'], hidden_size=cfg['hidden_size'], num_layers=[4,4,18,4], num_classes = 34)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=cfg['learning_rate'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=cfg['num_epochs'])

In [ ]:
# training
# kill process
# kill -9 PID_number

train_classification(model, train_loader, optimizer, scheduler, 
          save_interval=cfg['save_interval'],num_epochs=cfg['num_epochs'], 
          state_path=None,model_name='model1', device=device)

In [4]:
state = torch.load('./model_saves/model1_90')
model.load_state_dict(state['model'])

<All keys matched successfully>

In [17]:
x, t = next(iter(train_loader))
x = x.to(device)
out = model(x[:10])
out.size()
_, pred = out.max(dim=1)

In [18]:
t[:10]

tensor([ 8, 19, 30, 10, 25, 32,  5, 14, 19, 10])

In [19]:
pred

tensor([ 8, 19, 30, 10, 25, 32,  5, 14, 19, 10], device='cuda:0')